In [20]:
import yaml
YAML_LOADER = yaml.FullLoader
from tinydb import TinyDB, Query, where
from espei.utils import PickleableTinyDB, MemoryStorage
from espei.datasets import DatasetError, load_datasets
from pycalphad import Database, calculate, equilibrium, variables as v
from pycalphad.core.utils import filter_phases
from espei.core_utils import ravel_conditions
from pycalphad.plot.eqplot import _map_coord_to_variable
from pycalphad.core.calculate import instantiate_models
import numpy as np
import copy
from scipy.stats import norm
import logging

In [63]:
def calculate_Y_probability(dbf, comps, phases, datasets, parameters=None, phase_models=None, callables=None, data_weight=1.0):
    """
    Return the sum of square error from site fraction data

    Parameters
    ----------
    dbf : pycalphad.Database
        Database to consider
    comps : list
        List of active component names
    phases : list
        List of phases to consider
    datasets : espei.utils.PickleableTinyDB
        Datasets that contain single phase data
    parameters : dict
        Dictionary of symbols that will be overridden in pycalphad.equilibrium
    phase_models : dict
        Phase models to pass to pycalphad calculations
    callables : dict
        Callables to pass to pycalphad
    data_weight : float
        Weight for standard deviation of activity measurements, dimensionless.
        Corresponds to the standard deviation of differences in chemical
        potential in typical measurements of activity, in J/mol.

    Returns
    -------
    float
        A single float of the sum of square errors

    Notes
    -----
    General procedure:
    1. Get the datasets
    2. For each dataset
        a. Calculate current site fraction 
        b. Find the target site fraction
        c. Calculate error due to site fraction

    """
    std_dev = 500
    if parameters is None:
        parameters = {}
    error = 0
    Y_data=datasets.search(
        (where('output').test(lambda x: 'Y' in x)) &
        (where('components').test(lambda x: set(x).issubset(comps))))
    error = 0
    if len(Y_data) == 0:
        return error
    for ds in Y_data:
        data_comps=ds['components']
        species = list(map(v.Species, data_comps))
        data_phases = phases
        conditions = {}
        conds_list = [(cond, value) for cond, value in ds['conditions'].items() if cond not in ('P', 'T')]

        for comp_name, comp_x in conds_list:
            P, T, X = ravel_conditions(ds['values'], ds['conditions']['P'], ds['conditions']['T'], comp_x,Y=True)
            conditions[v.P] = P
            conditions[v.T] = T
            conditions[_map_coord_to_variable(comp_name)] = X
        conditions_list = [{c: conditions[c][i] for c in conditions.keys()} for i in range(len(conditions[v.T]))]
        current_Y = []
        model=instantiate_models(dbf, data_comps, ds['phases'])
        sublattice=model[ds['phases'][0]].site_fractions
        for conds in conditions_list:
            sample_eq_res = equilibrium(dbf, data_comps, data_phases, conds,
                                    model=phase_models, parameters=parameters,
                                        callables=callables)
        
            result_st=sample_eq_res.Y.where(sample_eq_res.Phase==ds['phases']).squeeze().values
            print(result_st)
            result = result_st[np.logical_not(np.isnan(result_st))]
            if len(result)==0 or len(result)<len(sublattice):
                return 
            elif len(result) > len(sublattice):
                result_st=result_st[0]
                result = result_st[np.logical_not(np.isnan(result_st))]
            print(result)
            current_Y=np.hstack((current_Y,result))
        target_Y=np.array(ds['values']).flatten()
        weight = ds.get('weight', 1.00)
        ind=[i for i,v in enumerate(target_Y) if v == None]
        target_Y=np.delete(target_Y,ind)
        current_Y=np.delete(current_Y,ind)
        pe =norm(loc=0, scale=0.1/(weight*data_weight)).logpdf(np.array(target_Y - current_Y, dtype=np.float64))
        print('pe=',pe)
        error += np.sum(pe)
        logging.debug('Site_fraction error - data: {}, site_fraction difference: {}, probability: {}, reference: {}'.format(target_Y, current_Y-target_Y, pe, ds["reference"]))

        # TODO: write a test for this
    if np.any(np.isnan(np.array([error], dtype=np.float64))):  # must coerce sympy.core.numbers.Float to float64
        return -np.inf
    return error

In [64]:
parameters={'VV0008': 219436.2359021377, 'VV0014': -159347.25647113487, 'VV0022': 7793.743163909789, 'VV0026': -115436.38652226927, 'VV0030': 46628.51147027806, 'VV0032': 704062.8690833185, 'VV0033': 532825.9201085522, 'VV0034': -224058.31679055002, 'VV0035': -231338.44317806137, 'VV0038': 137041.7029955661, 'VV1016': -125.75684118015415, 'VV1019': -89.1785694537837, 'VV1023': -241511.77861589784, 'VV1025': -188.9772772068905, 'VV1165': -401044.7640365149, 'VV1167': -679296.302666642, 'VV3028': -188223.55889780005, 'VV3030': -64.27771071281647, 'VV3033': -159737.11159104528, 'VV3045': -68312.73622893881, 'VV3047': 108327.87948478022, 'VV3067': -204655.94433847605, 'VV3068': 0.0, 'VV3167': -104736.92730932923, 'VV3168': -385961.5075697028, 'VV3169': -294017.5041766338, 'VV3269': 0.0, 'VV5007': 82168.77143927022}
parameters={'VV0008': 244853.47790127105, 'VV0014': -147354.6286255831, 'VV0022': 8172.900751994036, 'VV0026': -115491.64141462288, 'VV0030': 43099.73681708047, 'VV0032': 561359.9935705559, 'VV0033': 578970.6395314168, 'VV0034': -212249.55591059156, 'VV0035': -183911.010370692, 'VV0038': 123023.61576990165, 'VV1016': -108.29252785694246, 'VV1019': -97.6919961274951, 'VV1023': -208248.88521824882, 'VV1025': -181.5325644989351, 'VV1165': -482713.2562321738, 'VV1167': -621528.1365431971, 'VV3028': -168276.06907566413, 'VV3030': -77.96426578623408, 'VV3033': -178166.42698039976, 'VV3045': -80465.12853141097, 'VV3047': 110756.0504259704, 'VV3067': -215402.36995770983, 'VV3068': 0.0, 'VV3167': -82914.16538039465, 'VV3168': -319047.6884336362, 'VV3169': -237954.4375052136, 'VV3269': 0.0, 'VV5007': 94908.65833242374}
parameters={'VV0008': 214147.58861134585, 'VV0014': -125280.74726638556, 'VV0022': 7626.64388474024, 'VV0026': -123460.48662951564, 'VV0030': 52530.80144865542, 'VV0032': 676899.9719992308, 'VV0033': 522512.0664896596, 'VV0034': -210360.92083547747, 'VV0035': -199797.4020420086, 'VV0038': 145645.5530093004, 'VV1016': -103.9284754917935, 'VV1019': -106.61694893272413, 'VV1023': -231668.96473695026, 'VV1025': -151.94560892633837, 'VV1165': -402987.9954527444, 'VV1167': -607324.4145000202, 'VV3028': -195020.27443190565, 'VV3030': -66.27476535157314, 'VV3033': -149023.02500254242, 'VV3045': -104697.45861139549, 'VV3047': 77599.72819198355, 'VV3067': -186172.21313520902, 'VV3068': 0.0, 'VV3167': -106710.74614628183, 'VV3168': -407641.76047953183, 'VV3169': -250460.27163424683, 'VV3269': 0.0, 'VV5007': 105799.70391462307}
parameters={'VV0008': 220223.3135916236, 'VV0014': -137092.70503326348, 'VV0022': 6509.161711880144, 'VV0026': -116341.06433487819, 'VV0030': 47802.83989778641, 'VV0032': 663885.7441832282, 'VV0033': 551463.0991891087, 'VV0034': -204562.90060031405, 'VV0035': -263248.5532681535, 'VV0038': 128095.5968978, 'VV1016': -101.12960523867399, 'VV1019': -101.68387340002424, 'VV1023': -206324.5853096966, 'VV1025': -154.5741804037531, 'VV1165': -362071.52883893676, 'VV1167': -446390.5621566363, 'VV3028': -192879.56367645485, 'VV3030': -62.054144530443146, 'VV3033': -193703.84378549512, 'VV3045': -93974.43707691463, 'VV3047': 99846.808508017, 'VV3067': -201995.49930972315, 'VV3068': 0.0, 'VV3167': -89573.03557361403, 'VV3168': -363978.9010181534, 'VV3169': -271584.1572599032, 'VV3269': 0.0, 'VV5007': 82739.01967443136}
parameters={'VV0008': 2.35345676e+05, 'VV0014': -1.62248351e+05, 'VV0022': 1.05007547e+04, 'VV0026': -1.60104519e+05, 
            'VV0030': 2.75558761e+04, 'VV0032': 5.09959837e+05, 'VV0033': 8.81877650e+05, 'VV0034': -1.36102329e+05, 
            'VV0035': -4.79828734e+05, 'VV0038': 1.12892201e+05, 'VV1016': -1.12089328e+02, 'VV1019': -1.44594780e+02, 
            'VV1023': -2.72799019e+05, 'VV1025': -1.15400662e+02, 'VV1165': -4.01928624e+05, 'VV1167': -5.78246586e+05,
            'VV3028': -3.04041563e+05, 'VV3030': -4.60304253e+01, 'VV3033': -1.66342970e+05, 'VV3067': -2.32631019e+05, 'VV3167': -1.13980958e+05, 
            'VV3168': -4.71120845e+05, 'VV3169': -2.47417147e+05, 'VV5007': 1.04157736e+05}
parameters={'VV0008': 9.51290844e+04, 'VV0014': -1.20409581e+05, 'VV0022':  1.49979248e+04, 'VV0026':  -1.07753738e+04, 
           'VV0030':3.22862296e+04 , 'VV0032': -5.62721775e+04, 'VV0033': 2.17131072e+05, 'VV0034': -4.36245358e+05, 
            'VV0035':-2.97410362e+05, 'VV0038': 3.48833546e+05, 'VV1016': -3.55864276e+01, 'VV1019': -2.43552926e+01, 
            'VV1023': -2.24499095e+05, 'VV1025': 3.32651708e+01, 'VV1165': -5.46544824e+05, 'VV1167': -6.96236000e+05,
            'VV3028': -3.22246387e+05, 'VV3030': 3.43994356e+00, 'VV3033': -1.21622723e+05,  
             'VV3067': -1.28300070e+05, 'VV3167': -2.48113859e+05, 
            'VV3168': -3.61818898e+05, 'VV3169': -2.66969435e+05,  'VV5007': 6.81952604e+04}

dbf = Database('Nb-Ni-Y-now.TDB')
dataset=load_datasets(['input_Y_data-adj.json'])
comps=['NB', 'NI','VA']
phases = list(dbf.phases.keys())
a=calculate_Y_probability(dbf, comps, phases, dataset, parameters=parameters, phase_models=None, callables=None, data_weight=1.0)


[[           nan            nan            nan            nan
             nan            nan            nan            nan
             nan            nan]
 [3.75628760e-06 9.99996244e-01 9.32069892e-08 9.99999907e-01
  1.00000000e+00 8.93631494e-30 1.40396205e-01 8.59603795e-01
  9.99999995e-01 5.20839073e-09]
 [           nan            nan            nan            nan
             nan            nan            nan            nan
             nan            nan]]
[3.75628760e-06 9.99996244e-01 9.32069892e-08 9.99999907e-01
 1.00000000e+00 8.93631494e-30 1.40396205e-01 8.59603795e-01
 9.99999995e-01 5.20839073e-09]
[[           nan            nan            nan            nan
             nan            nan            nan            nan
             nan            nan]
 [3.75628760e-06 9.99996244e-01 9.32069892e-08 9.99999907e-01
  1.00000000e+00 5.38636763e-17 1.40396205e-01 8.59603795e-01
  9.99999995e-01 5.20839069e-09]
 [           nan            nan            nan            na

In [55]:
print(a_100)

-1777.289336062898


In [58]:
print(a_100)

-40237.45401542769


In [65]:
print(a)

-367.358149829477


In [52]:
print(a_1)

-24.07971499922502


In [42]:
print(a_1)

-27.925731467161498


In [46]:
print(a)

-367.358149829477


In [ ]:
-5885.155 -6449.378